**Práctica 8: REGRESIÓN LINEAL MÚLTIPLE Y
LOGÍSTICA**

In [41]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

1. Descargue el conjunto de datos “segment” del repositorio
“openml.org”. Se proporciona el origen del dataset
(http://archive.ics.uci.edu/ml/datasets/image+segmentation), aunque
lo que hay en el repositorio “uci”, no es exactamente igual que lo hay
en “openml”. A pesar de que está en diferentes formatos, quizás, el
más conveniente sea el arff de WEKA que, para ser manejado en
python junto sklearn, habrá que transformar a CSV, puesto que deberá
ser leído por el módulo pandas correspondiente.


In [3]:
segment=pd.read_csv('segment.csv')
segment

,region-centroid-col,region-centroid-row,region-pixel-count,short-line-density-5,short-line-density-2,vedge-mean,vegde-sd,hedge-mean,hedge-sd,intensity-mean,rawred-mean,rawblue-mean,rawgreen-mean,exred-mean,exblue-mean,exgreen-mean,value-mean,saturation-mean,hue-mean,class
0,218,178,9,0.111111,0.0,0.833333,0.547722,1.111110,0.544331,59.629600,52.44440,75.22220,51.222200,-21.555600,46.77780,-25.22220,75.22220,0.318996,-2.04055,path
1,113,130,9,0.000000,0.0,0.277778,0.250924,0.333333,0.365148,0.888889,0.00000,2.55556,0.111111,-2.666670,5.00000,-2.33333,2.55556,1.000000,-2.12325,foliage
2,202,41,9,0.000000,0.0,0.944448,0.772202,1.111110,1.025600,123.037000,111.88900,139.77800,117.444000,-33.444400,50.22220,-16.77780,139.77800,0.199347,-2.29992,sky
3,32,173,9,0.000000,0.0,1.722220,1.781590,9.000000,6.749490,43.592600,39.55560,52.88890,38.333300,-12.111100,27.88890,-15.77780,52.88890,0.266914,-1.99886,path
4,61,197,9,0.000000,0.0,1.444440,1.515350,2.611110,1.925460,49.592600,44.22220,61.55560,43.000000,-16.111100,35.88890,-19.77780,61.55560,0.302925,-2.02227,path
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,30,102,9,0.000000,0.0,1.222220,0.118518,1.333330,0.800000,20.259300,20.33330,25.00000,15.444400,0.222222,14.22220,-14.44440,25.00000,0.381059,-1.55510,brickface
2306,143,24,9,0.000000,0.0,1.277780,0.907406,0.888888,1.140750,127.630000,117.66700,141.66700,123.556000,-29.888900,42.11110,-12.22220,141.66700,0.169397,-2.34925,sky
2307,80,72,9,0.000000,0.0,1.222220,1.003700,1.444440,1.167460,59.000000,51.33330,74.44440,51.222200,-23.000000,46.33330,-23.33330,74.44440,0.314606,-2.09022,cement
2308,98,133,9,0.000000,0.0,0.555555,0.172133,0.388889,0.327731,0.962963,0.00000,2.77778,0.111111,-2.888890,5.44444,-2.55556,2.77778,1.000000,-2.12325,foliage


2. Con este fichero generado realice una clasificación mediante regresión
lineal múltiple. Para ello, se destinará ⅔ de los datos escogidos
aleatoriamente de manera estratificada para aprendizaje y, el resto,
para verificación.

In [5]:
scaler=MinMaxScaler()
X=scaler.fit_transform(segment.iloc[:,:-1])
clase=segment.iloc[:,-1].values

In [8]:
clase.shape, X.shape, np.min(X, axis=0), np.max(X, axis=0)

((2310,),
 (2310, 19),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]),
 array([1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.]))

Binarización de las Etiquetas (ONEHOT encoder)

In [15]:
YB = LabelBinarizer().fit_transform(clase)
YB

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

Método de HOLD-OUT (2/3 - 1/3)

In [40]:
X_train, X_test, Y_train, Y_test = train_test_split(X, YB, random_state = 1, stratify = YB, train_size = 2/3)

In [24]:
YB_predict = np.zeros(Y_test.shape)

for i in range(Y_test.shape[1]):
    regresion = LinearRegression().fit(X_train, Y_train[:,i])
    YB_predict[:,i] = regresion.predict(X_test)

In [25]:
print("Tasa de aciertos =", accuracy_score(np.argmax(Y_test, axis=1), np.argmax(YB_predict, axis=1)))

print("\nMatriz de Confusión:\n", confusion_matrix(np.argmax(Y_test, axis=1), np.argmax(YB_predict, axis=1)))

Tasa de aciertos = 0.8558441558441559

Matriz de Confusión:
 [[108   0   2   0   0   0   0]
 [ 16  47   7   0  28   2  10]
 [  1   1 100   0   0   0   8]
 [  0   0   0 108   2   0   0]
 [  0   0   0   0 110   0   0]
 [  0   0   0   0   0 110   0]
 [  0   2  30   0   2   0  76]]


3. Repetir el paso anterior empleando el método de validación cruzada
con estratificación. Compare los resultado.

In [32]:
kf = StratifiedKFold(n_splits=10)

scores_fold = np.empty(YB.shape[1])
scores_total = np.empty(kf.get_n_splits(X, clase))

j = 0
for train_idx, test_idx in kf.split(X, clase):
    X_train, X_test = X[train_idx],  X[test_idx]
    Y_train, Y_test = YB[train_idx], YB[test_idx]
    YB_predict = np.zeros(Y_test.shape)
    for i in range(YB.shape[1]):
        model = LinearRegression()
        model.fit(X_train, Y_train[:,i])
        YB_predict[:,i] = model.predict(X_test)
        scores_fold[i] = accuracy_score(np.argmax(YB_predict, axis=1), np.argmax(Y_test, axis=1))
        
    scores_total[j] = np.mean(scores_fold)
    j += 1

In [33]:
print(f'Tasa de acierto (Regresión lineal CV-10): {np.mean(scores_total)}')

Tasa de acierto (Regresión lineal CV-10): 0.5013605442176872


4. Repita los pasos 2 y 3 aplicando la Regresión logística. En ambos
casos, solamente cuando emplee el método del resorte o Hold-Out,
calcule la matriz de confusión.

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X, clase, stratify = clase, train_size = 2/3)

Y_predict = np.zeros(Y_test.shape)

model = LogisticRegression(max_iter=1000, n_jobs=-1) # 1K iteraciones para que converja
model.fit(X_train, Y_train)
Y_predict = model.predict(X_test)

In [36]:
print(f'Tasa de acierto: {model.score(X_test, Y_test)}')

print("\nMatriz de Confusión:\n", confusion_matrix(Y_predict, Y_test))

Tasa de acierto: 0.9

Matriz de Confusión:
 [[108   0   3   0   0   0   1]
 [  0  97   2   0   0   0  13]
 [  0   1  90   0   0   0  27]
 [  0   0   0 109   0   0   0]
 [  0   6   0   1 110   0   0]
 [  0   0   0   0   0 110   0]
 [  2   6  15   0   0   0  69]]


In [38]:
LR = LogisticRegression(n_jobs=-1, max_iter=1000)
cv_results = cross_validate(LR, X, clase, cv=10, n_jobs=-1)

In [39]:
score = np.mean(cv_results['test_score'])
print(f'Tasa de acierto (Regresión logística CV-10): {score}')

Tasa de acierto (Regresión logística CV-10): 0.9043290043290042
